In [37]:
import transformers
import sagemaker
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pathlib import Path

from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor
from datetime import datetime
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


In [21]:
sess = sagemaker.Session()
sagemaker_session_bucket = 'sagemaker-godeltech'
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

#put SageMaker role here if you're running this notebook locally
role = ''


sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker bucket: sagemaker-godeltech
sagemaker session region: eu-west-1


In [22]:
MODEL_NAME = 'unitary/toxic-bert'

In [23]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir = '../tmp/AutoTokenizer')  
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                           cache_dir = '../tmp/AutoModel',
                                                           return_dict=False)

In [24]:
# Create directory for model artifacts
Path("../tmp/traced_model/").mkdir(exist_ok=True)

In [25]:
# Prepare sample input for jit model tracing
seq = "Godel technologies: Sage Maker: this is just an example for PyTorch"
max_length = 512

tokenized_sequence_pair = tokenizer.encode_plus(
    seq, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt"
)

example = tokenized_sequence_pair["input_ids"], tokenized_sequence_pair["attention_mask"]

traced_model = torch.jit.trace(model.eval(), example)
traced_model.save("../tmp/traced_model/model.pth")

In [26]:
!tar -czvf ../tmp/traced_model.tar.gz -C ../tmp/traced_model . && mv ../tmp/traced_model.tar.gz ../tmp/traced_model/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
./
./model.pth
./traced_model.tar.gz


In [27]:
traced_model_url = sess.upload_data(
    path="../tmp/traced_model/traced_model.tar.gz",
    key_prefix="transformers/neuron-experiments/traced-model",
)

In [33]:
prefix = "neuron-experiments"
flavour = "normal"
date_string = datetime.now().strftime("%Y%m%d")

normal_sm_model = PyTorchModel(
    model_data=traced_model_url,
    predictor_cls=Predictor,
    framework_version="1.8.1",
    role=role,
    sagemaker_session=sess,
    entry_point="inference_normal.py",
    source_dir="./",
    py_version="py3",
    name=f"{flavour}-toxicbert-{date_string}",
    env={"SAGEMAKER_CONTAINER_LOG_LEVEL": "10"},
)

In [38]:
hardware = "g4dn"

normal_predictor = normal_sm_model.deploy(
    instance_type="ml.g4dn.xlarge",
    initial_instance_count=1,
    endpoint_name=f"toxicbert-{flavour}-{hardware}-{date_string}",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)
print('READY!')

-----------!READY!


In [41]:
payload = """according to contract invoices have to be payed within 15 day, no exceptions for public holidays were described. paying on Tuesday, 16th seems to be a breach of contract. Viktoryia Charnianina please correct me if I'm wrong"""
normal_predictor.predict(payload)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/toxicbert-normal-g4dn-202209-2611-4428 in account 798631296162 for more information.